In [1]:
from jupyter_plotly_dash import JupyterDash

import dash

import dash_core_components as dcc
import dash_html_components as html
import dash_daq as daq

import plotly.express as px
import dash_table
from dash.dependencies import Input, Output, State
import base64

import numpy as np
import pandas as pd
import dash_leaflet as dl
from pymongo import MongoClient

###########################
# Data Manipulation / Model
###########################
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from crud import AnimalShelter

# FIX ME update with your username and password and CRUD Python module name
username = "aacuser"
password = "HookRat94561"
shelter = AnimalShelter(username, password)

# readAll is the method name from AnimalShelter class and returns a cursor object
# update the next line to match your method name from your CRUD Python module class
df = pd.DataFrame.from_records(shelter.read({}, True))

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('Dash DataTable Only')

image_filename = 'GraziosoSalvareLogo.png'  # customer image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Center(html.B(html.H1('Austin Barnes CS-340 Dashboard'))),
    html.Hr(),
    # holder div for image logo and map feature
    html.Div(
        id='holder',
        style={'display': 'flex'},
        children=[
            html.Img(
        id='customer-image',
        style={'height': '500px'},
        src='data:image/png;base64,{}'.format(encoded_image.decode()),
        alt='customer image'
        ),
        html.Div(
        id='map-id',
        ),
        html.Div(
            id='graph-id',
            className='col s12 m6',

            )
        ]
    ),
    #########################
    # Toggles for filtering #
    #########################
        html.Div(
        #Radio Items to select the rescue filter options
        dcc.RadioItems(
            id='filter-type',
            # create the labels based on the Grazioso requirements
            options=[
            {'label': 'Water Rescue', 'value': 'WATER'},
            {'label': 'Mountain/Wilderness Rescue', 'value': 'WILD'},
            {'label': 'Disaster Rescue/Individual Tracking', 'value': 'DISASTER'}
            ],
            value=None,
            labelStyle={'display': 'inline-block'}
        )
    ),

    
   
    dash_table.DataTable(
        id='datatable-interactivity',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        style_header={
            'background': '#999999',
            'white-space': 'normal',
            'margin': 'auto',
            'vertical-align': 'bottom',
            'text-align': 'center',
        },
        style_data={
            'height': 'auto',
            'line-height': '15 px',
            'text-align': 'center',
        },
        style_data_conditional=[
            {
                'if': {'row_index': 'odd'},
                'backgroundColor': 'rgb(220, 220, 220)',
            }],
        data=df.to_dict('records'),
        editable=False,
        filter_action="none",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable=False,
        row_deletable=True,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=10,
    ),
    html.Br(),
    html.Hr(),
    
])

#############################################
# Interaction Between Components / Controller
#############################################

# use toggles to filter data between cats and dogs and delete deleted rows from db
@app.callback(Output('datatable-interactivity', 'data'),
             [Input('filter-type', 'value'),
              Input('datatable-interactivity', 'data_previous')],
              [State('datatable-interactivity', 'data')]
             )
def handle_datatable(filter_value, old_values, current_values):
    import os
    # takes rows deleted from the table and deletes them from the DB
    if old_values is not None:
        for row in old_values:
            if row not in current_values:
                shelter.delete({'animal_id': row['animal_id']})
    
    if filter_value == 'WATER':
        df = pd.DataFrame(list(shelter.read({'$and': [{'sex_upon_outcome': 'Intact Female'},
                                                          {'$or': [
                                                              {'breed': 'Labrador Retriever Mix'},
                                                              {'breed': 'Chesa Bay Retr Mix'},
                                                              {'breed': 'Newfoundland Mix'},
                                                              {'breed': 'Newfoundland/Labrador Retriever'},
                                                              {'breed': 'Newfoundland/Australian Cattle Dog'},
                                                              {'breed': 'Newfoundland/Great Pyrenees'}]
                                                          },
                                                          {'$and': [{'age_upon_outcome_in_weeks': {'$gte': 26}},
                                                                    {'age_upon_outcome_in_weeks': {'$lte': 156}}]
                                                          }]
                                                }, True)))
    elif filter_value == 'WILD':
        df = pd.DataFrame(list(shelter.read({'$and': [{'sex_upon_outcome': 'Intact Male'},
                                                          {'$or': [
                                                              {'breed': 'German Shepherd'},
                                                              {'breed': 'Alaskan Malamute'},
                                                              {'breed': 'Old English Sheepdog'},
                                                              {'breed': 'Rottweiler'},
                                                              {'breed': 'Siberian Husky'}]
                                                          },
                                                          {'$and': [{'age_upon_outcome_in_weeks': {'$gte': 26}},
                                                                    {'age_upon_outcome_in_weeks': {'$lte': 156}}]
                                                          }]
                                                }, True)))
    elif filter_value == 'DISASTER':
        df = pd.DataFrame(list(shelter.read({'$and': [{'sex_upon_outcome': 'Intact Male'},
                                                          {'$or': [
                                                              {'breed': 'Doberman Pinscher'},
                                                              {'breed': 'German Shepherd'},
                                                              {'breed': 'Golden Retriever'},
                                                              {'breed': 'Bloodhound'},
                                                              {'breed': 'Rottweiler'}]
                                                          },
                                                          {'$and': [{'age_upon_outcome_in_weeks': {'$gte': 20}},
                                                                    {'age_upon_outcome_in_weeks': {'$lte': 300}}]
                                                          }]
                                                }, True)))
    else:
        df = pd.DataFrame(list(shelter.read({}, True)))
    
    return df.to_dict('records')
    
    
# update map based on selected cell(row), if none selected shwo first row on map
@app.callback(Output('map-id', 'children'),
    [Input('datatable-interactivity', 'derived_viewport_data'),
     Input('datatable-interactivity', 'active_cell')
    ])
def update_map(viewData, active_cell):    
    if viewData is None:
        return None
    dff = pd.DataFrame(viewData)
    row = active_cell['row'] if active_cell else 0
    
    animal_type = dff.iloc[row, 3]
    name = dff.iloc[row, 9]
    breed = dff.iloc[row, 4]
    lat = dff.iloc[row, 13]
    lon = dff.iloc[row, 14]
    animal_id = dff.iloc[row, 2]
    age = dff.iloc[row, 1]
    color = dff.iloc[row, 5]
    sex = dff.iloc[row, 12]
    no_data = '(no data)'

    return [
        dl.Map(style={'width': '750px', 'height': '500px'}, center=[lat, lon], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[lat, lon], children=[
                dl.Tooltip(breed),
                dl.Popup([
                    html.H2(animal_id + ' - ' + animal_type),
                    html.Div(
                        id='data-holder',
                        style={'display': 'flex', 'justify-content': 'center'},
                        children=[
                            html.Div(
                                id='left',
                                style={
                                    'text-align': 'right'
                                      },
                                children=[
                                    html.H3('Name: '),
                                    html.H3('Age: '),
                                    html.H3('Breed: '),
                                    html.H3('Color: '),
                                    html.H3('Sex: '),
                                ]
                            ),
                            html.Div(
                                id='right',
                                children=[
                                    html.H3(name if name else no_data),
                                    html.H3(age if age else no_data),
                                    html.H3(breed if breed else no_data),
                                    html.H3(color if color else no_data),
                                    html.H3(sex if sex else no_data),
                                ]
                            ),
                        ]
                    ),
                ])
            ])
        ])
    ]

# updates graph
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-interactivity', "derived_viewport_data")])
def update_graphs(viewData):

    dff = pd.DataFrame.from_dict(viewData)
    names = dff['breed'].value_counts().keys().tolist()
    values = dff['breed'].value_counts().tolist()
    #creates a pie chart based on the data above
    return [
        dcc.Graph(            
            figure = px.pie(
                data_frame=dff, 
                values = values, 
                names = names, 
                color_discrete_sequence=px.colors.sequential.RdBu,
                width=600, 
                height=500   
            )
        )
    ]


app

/home/leftii/PycharmProjects/mongocrud/venv/lib/python3.6/site-packages/pymongo/pyopenssl_context.py:26: CryptographyDeprecationWarning:

Python 3.6 is no longer supported by the Python core team. Therefore, support for it is deprecated in cryptography and will be removed in a future release.

